<a href="https://colab.research.google.com/github/ralsouza/apache_spark_real_time_analytics/blob/master/notebooks/spark_streaming_twitter/01_spark_streaming_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark Setup

In [ ]:
!apt-get update

In [3]:
# Install the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
# Environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
# Libraries and Context Setup
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)

# Instance Spark Session
spark = SparkSession.builder.master('local').appName('My-SparkSQL').getOrCreate()

# Create the SQL Context
sqlContext = pyspark.SQLContext(sc)

In [6]:
# Check context
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


## Other packages to streaming - Twitter

In [ ]:
!pip install requests_oauthlib
!pip install twython
!pip install nltk

## Install Modules

In [9]:
from pyspark.streaming import StreamingContext
from requests_oauthlib import OAuth1Session
from operator import add
from time import gmtime, strftime
import requests
import time
import string 
import ast

## Install NLTK modules

In [11]:
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [13]:
# Update frequency
BATCH_INTERVAL = 5

In [14]:
# Making the StreamingContext
ssc = StreamingContext(sc,batchDuration=BATCH_INTERVAL)